## NLP Project #1 for LEARNING

In [1]:
import nltk
#nltk.download()
import numpy as np

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [2]:
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from bs4 import BeautifulSoup

Lemmatization is the process of grouping together the different inflected forms of a word so they can be analysed as a single item. Lemmatization is similar to stemming but it brings context to the words. So it links words with similar meaning to one word.

Text preprocessing includes both Stemming as well as Lemmatization. Many times people find these two terms confusing. Some treat these two as same. Actually, lemmatization is preferred over Stemming because lemmatization does morphological analysis of the words.

In [3]:
# Lemmatize 
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [4]:
# quick examples
print("rocks :", lemmatizer.lemmatize("rocks")) 
print("corpora :", lemmatizer.lemmatize("corpora"))

rocks : rock
corpora : corpus


In [5]:
# from http://www.lextek.com/manuals/onix/stopwords1.html
stopwords = set(w.rstrip() for w in open('D:\Data\stopwords.txt'))

# from nltk.corpus import stopwords
# stop_words = set(stopwords.words('english')) 

https://beautiful-soup-4.readthedocs.io/en/latest/

Beautiful Soup is a Python library for pulling data out of HTML and XML files. It works with your favorite parser to provide idiomatic ways of navigating, searching, and modifying the parse tree. It commonly saves programmers hours or days of work.

In [6]:
# load the reviews
# data courtesy of http://www.cs.jhu.edu/~mdredze/datasets/sentiment/index2.html
positive_reviews = BeautifulSoup(open(r'D:\Data\sorted_data_acl\electronics\positive.review').read(), features="html5lib")
positive_reviews = positive_reviews.findAll('review_text')
negative_reviews = BeautifulSoup(open(r'D:\Data\sorted_data_acl\electronics\negative.review').read(), features="html5lib")
negative_reviews = negative_reviews.findAll('review_text')

In [18]:
type(positive_reviews)

bs4.element.ResultSet

Tokenization is a common task in Natural Language Processing (NLP). It’s a fundamental step in both traditional NLP methods like Count Vectorizer and Advanced Deep Learning-based architectures like Transformers.

Tokens are the building blocks of Natural Language.

Tokenization is a way of separating a piece of text into smaller units called tokens. Here, tokens can be either words, characters, or subwords. Hence, tokenization can be broadly classified into 3 types – word, character, and subword (n-gram characters) tokenization.

In [14]:
# take a look at a review
t = positive_reviews[0]
t

<review_text>
I purchased this unit due to frequent blackouts in my area and 2 power supplies going bad.  It will run my cable modem, router, PC, and LCD monitor for 5 minutes.  This is more than enough time to save work and shut down.   Equally important, I know that my electronics are receiving clean power.

I feel that this investment is minor compared to the loss of valuable data or the failure of equipment due to a power spike or an irregular power supply.

As always, Amazon had it to me in &lt;2 business days
</review_text>

tokenize vs split

tokenize() ,which returns a list, will ignore empty string (when a delimiter appears twice in succession) where as split() keeps such string. The split() can take regex as delimiter where as tokenize does not

In [15]:
# first let's just try to tokenize the text using nltk's tokenizer
from nltk.stem import WordNetLemmatizer
print(nltk.tokenize.word_tokenize(t.text))

['I', 'purchased', 'this', 'unit', 'due', 'to', 'frequent', 'blackouts', 'in', 'my', 'area', 'and', '2', 'power', 'supplies', 'going', 'bad', '.', 'It', 'will', 'run', 'my', 'cable', 'modem', ',', 'router', ',', 'PC', ',', 'and', 'LCD', 'monitor', 'for', '5', 'minutes', '.', 'This', 'is', 'more', 'than', 'enough', 'time', 'to', 'save', 'work', 'and', 'shut', 'down', '.', 'Equally', 'important', ',', 'I', 'know', 'that', 'my', 'electronics', 'are', 'receiving', 'clean', 'power', '.', 'I', 'feel', 'that', 'this', 'investment', 'is', 'minor', 'compared', 'to', 'the', 'loss', 'of', 'valuable', 'data', 'or', 'the', 'failure', 'of', 'equipment', 'due', 'to', 'a', 'power', 'spike', 'or', 'an', 'irregular', 'power', 'supply', '.', 'As', 'always', ',', 'Amazon', 'had', 'it', 'to', 'me', 'in', '<', '2', 'business', 'days']


In [23]:
# notice how it doesn't downcase, so It != it
# not only that, but do we really want to include the word "it" anyway?
# you can imagine it wouldn't be any more common in a positive review than a negative review
# so it might only add noise to our model.
# so let's create a function that does all this pre-processing for us

def my_tokenizer(s):
    tokens = nltk.tokenize.word_tokenize(s.lower()) # split string into words (tokens)
    tokens = [t for t in tokens if len(t) > 2] # remove short words, they're probably not useful
    tokens = [lemmatizer.lemmatize(t) for t in tokens] # put words into base form
    tokens = [t for t in tokens if t not in stopwords] # remove stopwords
    return tokens

In [79]:
# create a word-to-index map so that we can create our word-frequency vectors later
# let's also save the tokenized versions so we don't have to tokenize again later
word_index_map = {}
current_index = 0
positive_tokenized = []
negative_tokenized = []
original_reviews = []

def word_index_map_filler (review_ResultSet, current_index): 
    # word counter
    # positive_review is a bs4.element.ResultSet
    for review in review_ResultSet:
        original_reviews.append(review.text)
        tokens = my_tokenizer(review.text)
        positive_tokenized.append(tokens)
        for token in tokens:
            if token not in word_index_map:
                word_index_map[token] = current_index
                current_index += 1

word_index_map_filler(positive_reviews, current_index)
word_index_map_filler(negative_reviews, current_index)

In [80]:
for review in positive_reviews:
    original_reviews.append(review.text)
    tokens = my_tokenizer(review.text)
    positive_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index += 1

for review in negative_reviews:
    original_reviews.append(review.text)
    tokens = my_tokenizer(review.text)
    negative_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index += 1

In [93]:
print("len(word_index_map):", len(word_index_map))

len(word_index_map): 11082


In [82]:
# now let's create our input matrices
def tokens_to_vector(tokens, label):
    x = np.zeros(len(word_index_map) + 1) # last element is for the label
    for t in tokens:
        i = word_index_map[t]
        x[i] += 1
    x = x / x.sum() # normalize it before setting label
    x[-1] = label
    return x

In [83]:
N = len(positive_tokenized) + len(negative_tokenized)
# (N x D+1 matrix - keeping them together for now so we can shuffle more easily later
data = np.zeros((N, len(word_index_map) + 1))
i = 0
for tokens in positive_tokenized:
    xy = tokens_to_vector(tokens, 1)
    data[i,:] = xy
    i += 1

for tokens in negative_tokenized:
    xy = tokens_to_vector(tokens, 0)
    data[i,:] = xy
    i += 1

In [87]:
data.size

22166000

In [90]:
# shuffle the data and create train/test splits
# try it multiple times!
original_reviews, data = shuffle(original_reviews, data)

X = data[:,:-1]
Y = data[:,-1]

# last 100 rows will be test
Xtrain = X[:-100,]
Ytrain = Y[:-100,]
Xtest = X[-100:,]
Ytest = Y[-100:,]

model = LogisticRegression()
model.fit(Xtrain, Ytrain)
print("Train accuracy:", model.score(Xtrain, Ytrain))
print("Test accuracy:", model.score(Xtest, Ytest))

C:\Users\ytjya\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Train accuracy: 0.7736842105263158
Test accuracy: 0.76


In [104]:
# let's look at the weights for each word
# try it with different threshold values!
threshold = 0.5
for word, index in word_index_map.items():
    weight = model.coef_[0][index]
    if weight > threshold or weight < -threshold:
        print(word, weight)

unit -0.6550971099533333
bad -0.7849407210705378
cable 0.5801306870325602
time -0.544523090353267
've 0.7957072077930096
month -0.7533632856858961
sound 1.0882875689921967
lot 0.7456031131815123
you 0.8557737579280181
n't -2.1542038593220236
easy 1.7743269565224105
quality 1.456715953872191
company -0.5554719087880162
item -0.9129136325144119
wa -1.6789345805691982
perfect 0.9490018599729955
fast 0.9173785105050907
ha 0.6859594066279514
price 2.609573710768579
value 0.5616259246403541
money -1.0452726173907554
memory 0.930704082657109
picture 0.5063567942732812
buy -0.9080229682060375
bit 0.6784065417732267
happy 0.6215205318288972
pretty 0.7343964402329317
doe -1.2195353512100804
highly 0.9679739017035716
recommend 0.6646476175884155
customer -0.6812308185025778
support -0.8357215827598483
little 0.984985535456441
returned -0.8095779053945313
excellent 1.4101462192217555
love 1.055818949581985
home 0.5234355714572507
week -0.7396005172883073
using 0.6538769233256019
laptop 0.545123614

In [107]:
# check misclassified examples
preds = model.predict(X)
P = model.predict_proba(X)[:,1] # p(y = 1 | x)

# since there are many, just print the "most" wrong samples
minP_whenYis1 = 1
maxP_whenYis0 = 0
wrong_positive_review = None
wrong_negative_review = None
wrong_positive_prediction = None
wrong_negative_prediction = None
for i in range(N):
    p = P[i]
    y = Y[i]
    if y == 1 and p < 0.5:
        if p < minP_whenYis1:
            wrong_positive_review = original_reviews[i]
            wrong_positive_prediction = preds[i]
            minP_whenYis1 = p
    elif y == 0 and p > 0.5:
        if p > maxP_whenYis0:
            wrong_negative_review = original_reviews[i]
            wrong_negative_prediction = preds[i]
            maxP_whenYis0 = p

print("Most WRONG positive review (prob = %s, pred = %s):" % (minP_whenYis1, wrong_positive_prediction))
print(wrong_positive_review)
print("Most WRONG negative review (prob = %s, pred = %s):" % (maxP_whenYis0, wrong_negative_prediction))
print(wrong_negative_review)

Most WRONG positive review (prob = 0.3544260199338042, pred = 0.0):

I didn't buy this on Amazon but wanted to say this device is great. The only bad thing was MY laptop is old!  Can't go wrong with this one

Most WRONG negative review (prob = 0.606156367265541, pred = 1.0):

The Voice recorder meets all my expectations and more
Easy to use, easy to transfer great results

